In [1]:
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn import preprocessing

In [2]:
df = pd.read_csv('CC GENERAL.csv')

In [3]:
df = df.set_index('CUST_ID')

In [4]:
df = df.dropna() #Dropping any null values

## Normalizing Values

In [5]:
val = df.values
#Normailizing all column values using sklearn MinMaxScaler
min_max_scalar = preprocessing.MinMaxScaler()
df_scaled = min_max_scalar.fit_transform(val)
df = pd.DataFrame(df_scaled, columns=df.columns)

In [6]:
df.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000,8636.000000
mean,0.084084,0.895035,0.020910,0.014840,0.018704,0.021091,0.496000,0.205909,0.368820,0.091736,0.026942,0.041992,0.149319,0.035181,0.011312,0.159304,0.922398
std,0.110043,0.207697,0.044191,0.041321,0.040766,0.045006,0.401273,0.300054,0.398093,0.134528,0.056199,0.070337,0.122178,0.057368,0.031052,0.296271,0.218497
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.007777,0.909091,0.000884,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.002793,0.051753,0.008251,0.002214,0.000000,1.000000
50%,0.048146,1.000000,0.007655,0.001104,0.004213,0.000000,0.500000,0.083333,0.166667,0.000000,0.000000,0.019553,0.098497,0.017677,0.004089,0.000000,1.000000
75%,0.110549,1.000000,0.023368,0.014698,0.021518,0.024023,0.916667,0.333333,0.750000,0.166667,0.032520,0.050279,0.215359,0.038467,0.010804,0.166667,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
np.random.seed(42) #Random Seed for getting similar products

## Greedy K_Center Algorithm

In [8]:
def k_center(df, k):
    centers = []
    centers.append(np.random.randint(0, df.shape[0]-1))
    for _ in range(k):
        dist = [np.inf]*df.shape[0]
        for index, ctr in enumerate(centers):
            df_t = df.subtract(df.iloc[ctr])
            dist_t = np.linalg.norm(df_t, axis=1)
            dist = [min(d1, d2) for d1, d2 in zip(dist, dist_t)]
        print("Iteraion ", _+1, ": ", max(dist))
        if _ == k-1:
            print("Cost of Clustering: ", max(dist))
            break
        new_ctr = dist.index(max(dist))
        centers.append(new_ctr)

## Running Greedy Algorithm for K-Center for k = 2, 4, 10

In [16]:
k_center(df, 2)

Iteraion  1 :  2.7006424801270006
Iteraion  2 :  2.0387273608257246
Cost of Clustering:  2.0387273608257246


In [17]:
k_center(df, 4)

Iteraion  1 :  2.7126722169692945
Iteraion  2 :  2.0362025604304015
Iteraion  3 :  1.9399728833856422
Iteraion  4 :  1.9130838063058382
Cost of Clustering:  1.9130838063058382


In [9]:
k_center(df, 10)

Iteraion  1 :  2.265197996078704
Iteraion  2 :  2.0373563608155822
Iteraion  3 :  1.9864849418166526
Iteraion  4 :  1.8808492218050927
Iteraion  5 :  1.8146989047056628
Iteraion  6 :  1.6866406471556263
Iteraion  7 :  1.5902928177829083
Iteraion  8 :  1.5314195616154698
Iteraion  9 :  1.5188841649071865
Iteraion  10 :  1.4984577639954084
Cost of Clustering:  1.4984577639954084


In [19]:
df_small = df.iloc[:20] #Small DataSet

In [11]:
def calc_cost(df, centers):
    dist = [np.inf]*df.shape[0]
    for index, ctr in enumerate(centers):
        df_t = df.subtract(df.iloc[ctr])
        dist_t = np.linalg.norm(df_t, axis=1)
        dist = [min(d1, d2) for d1, d2 in zip(dist, dist_t)]
    return max(dist)

## Optimal Algorithm for K-Center, K = 2

In [12]:
#Optimal Cluster for k = 2
min_cost_k_2 = np.inf
for i in range(df_small.shape[0]):
    for j in range(df_small.shape[0]):
        centers = [i, j]
        min_cost_k_2 = min(min_cost_k_2, calc_cost(df_small, centers))
print("Cost of Clustering: ", min_cost_k_2)

Cost of Clustering:  1.1386643594179626


In [13]:
k_center(df_small, 2)

Iteraion  1 :  1.651154936670848
Iteraion  2 :  1.2554417392935193
Cost of Clustering:  1.2554417392935193


In [22]:
print("Approximation Factor for K = 2:", 1.2554417392935193/1.1386643594179626)

Approximation Factor for K = 2: 1.1025564547706124


## Optimal Algorithm for K-Center, K = 4

In [14]:
#Optimal Cluster for k = 4
min_cost_k_4 = np.inf
for i in tqdm(range(df_small.shape[0])):
    for j in range(i+1, df_small.shape[0]):
        for k in range(j+1, df_small.shape[0]):
            for l in range(k+1, df_small.shape[0]):
                centers = [i, j, k, l]
                min_cost_k_4 = min(min_cost_k_4, calc_cost(df_small, centers))
print("Cost of Clustering: ", min_cost_k_4)

100%|██████████| 20/20 [00:06<00:00,  3.23it/s]

Cost of Clustering:  0.7051403155666904


In [15]:
k_center(df_small, 4)

Iteraion  1 :  1.836471418744053
Iteraion  2 :  1.3095439421253219
Iteraion  3 :  1.096455100184628
Iteraion  4 :  0.7835629464950162
Cost of Clustering:  0.7835629464950162


In [23]:
print("Approximation Factor for K = 4:", 0.7835629464950162/0.7051403155666904)

Approximation Factor for K = 4: 1.1112156392097663
